In [1]:
import os
import nltk
import pandas
import collections
import itertools
import numpy as np

In [2]:
K = 0.1
POS_PATH = './trainData/posTrain/'
NEG_PATH = './trainData/negTrain/'
POS_TEST_PATH = './TestData/posTest/'
NEG_TEST_PATH = './TestData/negTest/'

In [3]:
tokenizer = nltk.tokenize.RegexpTokenizer(r'[a-zA-Z]+[\']*[a-zA-Z]+|[;!?$]')

In [4]:
# Get all the training files.. this will take a sec
posFileList = [POS_PATH+f for f in os.listdir(POS_PATH)]
negFileList = [NEG_PATH+f for f in os.listdir(NEG_PATH)]

In [5]:
posTestList = [POS_TEST_PATH+f for f in os.listdir(POS_TEST_PATH)]
negTestList = [NEG_TEST_PATH+f for f in os.listdir(NEG_TEST_PATH)]

In [20]:
# Input fileList = list of files in directory with all tokens of one class
# Returns tokens - list of words in all files. 
def getTokens(fileList):
    tokens = []
    for filename in fileList:
        with open(filename, 'r', encoding='ISO-8859-1') as f:
            rawText = f.read()
            tokens += tokenizer.tokenize(rawText)
            
    return tokens


In [26]:
posTokens = getTokens(posFileList)
negTokens = getTokens(negFileList)

In [8]:
conditionalProbPos = {}
conditionalProbNeg = {}
# Calculate smoothing and prior

In [20]:
TP = sum(resultsP.values())
FN = len(resultsP.values()) - TP
TN = sum(resultsN.values())
FP = len(resultsN.values()) - TN
acc = (TP+TN)/(TP+FN+FP+TN)
print('Pred \ Gold \t P \t N')
print('P \t \t '+str(TP)+' \t '+str(FP))
print('N \t \t '+str(FN)+' \t '+str(TN))

Pred \ Gold 	 P 	 N
P 	 	 73 	 21
N 	 	 16 	 69


In [19]:
acc = (TP+TN)/float(TP+FN+FP+TN)
print('accuracy is: ')
print(acc)

accuracy is: 
0.793296089385
